In [ ]:
!pip install Arabic-Stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 353 kB 5.0 MB/s 
     |████████████████████████████████| 126 kB 37.7 MB/s 


In [ ]:
from sklearn.metrics import classification_report ,confusion_matrix ,accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import TreebankWordTokenizer
from keras.preprocessing.text import Tokenizer
import arabicstopwords.arabicstopwords as stp
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
import collections
import numpy as np
import matplotlib
import argparse
import codecs
import string
import pickle
import nltk
import math 
import sys
import re
import os
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/task_1'
data_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/task_1/DataSets'
results_path = '/content/drive/MyDrive/Colab Notebooks/Tasks/task_1/results'

# Dataset Preparation (pre-processing)

In [ ]:
# data = pd.DataFrame(pd.read_excel(os.path.join(data_path,"Copy of FullArabicData.xlsx"))).to_csv(os.path.join(data_path,"Copy of FullArabicData.xlsx"),index=False)

In [ ]:
# data = pd.read_csv(os.path.join(data_path,"Copy of FullArabicData.csv"))
data = pd.read_csv(os.path.join(data_path,'IslamWebAR&FullArabicData.csv'))

In [ ]:
print(data.shape)
data.head()

(43771, 2)


,Question,Main Topic
0,هل هذا حديث يجوز الاستشهاد به قال رسول الله صل...,الحديث وعلومه
1,ما صحة قراءة خمس آيات صباحا ومساء فيها خمسون ق...,الأذكار والأدعية
2,مات أبي ولم يكن له أبناء غيري (أنا أنثى بالغة)...,فقه المواريث
3,أنا طالبة في الثانوية العامة، أنعم عليّ الله س...,الآداب والأخلاق والرقائق
4,بسم الله الرحمن الرحيمالسؤال/ من ماذا خلق الله...,الأطعمة والأشربة والصيد


In [ ]:
# data.drop(['Answer','Summary','Contents','Date','Views'],axis=1,inplace=True)

In [ ]:
data.head()

,Question,Main Topic
0,هل هذا حديث يجوز الاستشهاد به قال رسول الله صل...,الحديث وعلومه
1,ما صحة قراءة خمس آيات صباحا ومساء فيها خمسون ق...,الأذكار والأدعية
2,مات أبي ولم يكن له أبناء غيري (أنا أنثى بالغة)...,فقه المواريث
3,أنا طالبة في الثانوية العامة، أنعم عليّ الله س...,الآداب والأخلاق والرقائق
4,بسم الله الرحمن الرحيمالسؤال/ من ماذا خلق الله...,الأطعمة والأشربة والصيد


In [ ]:
data['Main Topic'].value_counts()

الفقه وأصوله                14471
الآداب والأخلاق والرقائق    11238
فقه الأسرة                   3145
العقيدة                      2645
القرآن الكريم                1919
الدعوة ووسائلها              1718
الحديث وعلومه                1631
فقه المواريث                 1616
القرآن وعلومه                 899
العلم والدعوة                 695
التاريخ والسيرة               496
الأذكار والأدعية              496
تراجم وشخصيات                 381
الفضائل والتراجم              364
مشكلات نفسية واجتماعية        351
الأطعمة والأشربة والصيد       342
مصادر الفقه الإسلامي          314
أصول الفقه وقواعده            277
الأقضية والشهادات             267
العقيدة الإسلامية             106
التربية                        93
فكر وسياسة وفن                 70
فقه المعاملات                  48
السياسة الشرعية                42
الحديث الشريف                  39
السيرة النبوية                 38
طب وإعلام وقضايا معاصرة        26
فقه الأسرة المسلمة             14
الحدود والتعزيرات              10
فقه العبادات  

In [ ]:
# 1616 
print(len(data['Main Topic'].tolist()))
list_of_label = list(set(data['Main Topic'].tolist()))
len(list_of_label) 

43771


34

In [ ]:
def isNaN(string):
    if string == string:
        return string
    return "nan"

In [ ]:
data['Question'] = data['Question'].apply(isNaN)
data[data['Question']=='nan'].shape

(0, 2)

In [ ]:
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [ ]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

In [ ]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def clean_text(text):
    text = remove_diacritics(text)
    text = remove_punctuations(text)
    text = remove_repeating_char(text)
    text = re.sub(r"([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+", " ", text)
    text = " ".join([word for word in text.split() if not stp.is_stop(u"{}".format(word))])
    text = " ".join([word for word in text.split() if len(word)>2])
    return text

In [ ]:
data['Question'] = data['Question'].apply(clean_text)

In [ ]:
data.head(3)

,Question,Main Topic
0,حديث يجوز الاستشهاد قال رسول اله صلى اله وسلم ...,الحديث وعلومه
1,صحة قراءة خمس آيات صباحا خمسون قافا لتحصين وال...,الأذكار والأدعية
2,مات يكن أبناء أنثى بالغةورثته وعمي الأول شقيق ...,فقه المواريث


In [ ]:
def marg(text):
    return " ".join([sen for sen in text]) if type(text) is list else text

In [ ]:
X_train = []
y_train = []

X_test = []
y_test = []

labels = []

for label in list_of_label:
    if len(data[data['Main Topic']==label]['Question'].tolist())<1614:
      continue
    x = []
    y = []
    labels.append(label)
    xT1 = data[data['Main Topic']==label]['Question'].tolist()
    yT1_ = data[data['Main Topic']==label]['Main Topic'].tolist()
    yT1 = [1 for _ in range(len(yT1_))]
    for l in list_of_label:
        if l == label:
            continue
        LENGTH = len(data[data['Main Topic']==l]['Question'].tolist())
        p = math.ceil((((len(xT1)*100)/data.shape[0])/100)*LENGTH) 
        xT2 = data[data['Main Topic']==l]['Question'].tolist()[:p]
        yT2_ = data[data['Main Topic']==l]['Main Topic'].tolist()[:p]
        x.append(xT2)
        y.append([0 for _ in range(len(yT2_))])
    xT2 = list(np.concatenate(x).flat)
    yT2 = list(np.concatenate(y).flat)
    TX_train , TX_test , Ty_train , Ty_test = train_test_split(list(np.concatenate([xT1,xT2]).flat),
                                                           np.array(list(np.concatenate([yT1,yT2]).flat)),
                                                           test_size=0.10,random_state=1000)
    X_train.append(TX_train)
    y_train.append(Ty_train)
    X_test.append(TX_test)
    y_test.append(Ty_test)

In [ ]:
len(X_train)

8

In [ ]:
for i,y in enumerate(y_train):
    print("=================","for data :",labels[i],"==================")
    print(len(y))
    elements_count = collections.Counter(y)
    for key, value in elements_count.items():
        print(f"{key}: {value}")

================= for data : الفقه وأصوله ==================
21755
0: 8729
1: 13026
================= for data : القرآن الكريم ==================
3393
1: 1720
0: 1673
================= for data : الحديث وعلومه ==================
2897
0: 1443
1: 1454
================= for data : فقه المواريث ==================
2871
0: 1419
1: 1452
================= for data : العقيدة ==================
4631
0: 2271
1: 2360
================= for data : فقه الأسرة ==================
5473
1: 2834
0: 2639
================= for data : الدعوة ووسائلها ==================
3048
1: 1540
0: 1508
================= for data : الآداب والأخلاق والرقائق ==================
17646
1: 10107
0: 7539


In [ ]:
for i,y in enumerate(y_test):
    print("=================","for data :",labels[i],"==================")
    print(len(y))
    elements_count = collections.Counter(y)
    for key, value in elements_count.items():
        print(f"{key}: {value}")

================= for data : الفقه وأصوله ==================
2418
0: 973
1: 1445
================= for data : القرآن الكريم ==================
377
1: 199
0: 178
================= for data : الحديث وعلومه ==================
322
1: 177
0: 145
================= for data : فقه المواريث ==================
319
0: 155
1: 164
================= for data : العقيدة ==================
515
0: 230
1: 285
================= for data : فقه الأسرة ==================
609
1: 311
0: 298
================= for data : الدعوة ووسائلها ==================
339
0: 161
1: 178
================= for data : الآداب والأخلاق والرقائق ==================
1961
1: 1131
0: 830


In [ ]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [ ]:
def tfidf_vectorizer(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

In [ ]:
def count_vectorizer(data): # Bag of words (BOW)  , binary=True
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

In [ ]:
def pad_sequences_vectorizer(data):
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(data)
    data = tokenizer.texts_to_sequences(data)
    #vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
    data = pad_sequences(data, padding='post', maxlen=10000)
    return data,tokenizer

In [ ]:
def train_models(X_train, y_train,save_model,labels):
    # knn = KNeighborsClassifier(n_neighbors= 10).fit(X_train, y_train)
    # save_model['KNeighborsClassifier'][labels].append(knn)
    # lr = LogisticRegression().fit(X_train, y_train)
    # save_model['LogisticRegression'][labels].append(lr)
    # dt = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
    # save_model['DecisionTreeClassifier'][labels].append(dt)
    svm = svm = SVC(kernel = 'rbf',probability=True).fit(X_train, y_train)
    save_model['SVM'][labels].append(svm)
    return svm # knn,lr,dt,

In [ ]:
def evaluate_result(model,X_test,y_test,model_name,label,Container_Result):
    print("==========",model_name,"==========")
    y_predict = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    Container_Result[model_name][label].append(y_proba)
    confusion = confusion_matrix(y_test, y_predict)
    print_cm(confusion, ['Not one', 'One'])
    print(classification_report(y_test, y_predict, target_names=['Not one', 'One']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_predict)))
    print('Precision: {:.2f}'.format(precision_score(y_test, y_predict)))
    print('Recall: {:.2f}'.format(recall_score(y_test, y_predict)))
    print('F1: {:.2f}'.format(f1_score(y_test, y_predict)))

In [ ]:
def Creat_Container_Result():
    Container_Result = {}
    Container_Result['KNeighborsClassifier'] = {}
    Container_Result['LogisticRegression'] = {}
    Container_Result['DecisionTreeClassifier'] = {}
    Container_Result['SVM'] = {}
    for label in labels:
        Container_Result['KNeighborsClassifier'][label] = []
        Container_Result['LogisticRegression'][label] = []
        Container_Result['DecisionTreeClassifier'][label] = []
        Container_Result['SVM'][label] = []
    return Container_Result

In [ ]:
# .predict_proba(X_test)
# .decision_function(X_test)

In [ ]:
labels

['الفقه وأصوله',
 'القرآن الكريم',
 'الحديث وعلومه',
 'فقه المواريث',
 'العقيدة',
 'فقه الأسرة',
 'الدعوة ووسائلها',
 'الآداب والأخلاق والرقائق']

### with TFIDF vectorizer 

In [ ]:
save_model = {}
save_model['KNeighborsClassifier'] = {}
save_model['LogisticRegression'] = {}
save_model['DecisionTreeClassifier'] = {}
save_model['SVM'] = {}
for i in labels:
    save_model['KNeighborsClassifier'][i] = []
    save_model['LogisticRegression'][i] = []
    save_model['DecisionTreeClassifier'][i] = []
    save_model['SVM'][i] = []

In [ ]:
TFIDF_Container_Result = Creat_Container_Result()
vectorizers = []
for i in range(len(X_train)):
    print("*******************************************************************************************")
    print("******************************","result for data",labels[i],"******************************")
    print("*******************************************************************************************")
    print("vectorizeation ....")
    X_tr , vectorizer = tfidf_vectorizer(X_train[i])
    vectorizers.append(vectorizer)
    X_te = vectorizer.transform(X_test[i])
    print("training .....")
    #knn,lr,dt,
    svm = train_models(X_tr,y_train[i],save_model,labels[i])
    print('evaluation ....')
    # evaluate_result(knn,X_te,y_test[i],'KNeighborsClassifier',labels[i],TFIDF_Container_Result)
    # evaluate_result(lr,X_te,y_test[i],'LogisticRegression',labels[i],TFIDF_Container_Result)
    # evaluate_result(dt,X_te,y_test[i],'DecisionTreeClassifier',labels[i],TFIDF_Container_Result)
    evaluate_result(svm,X_te,y_test[i],'SVM',labels[i],TFIDF_Container_Result)

*******************************************************************************************
****************************** result for data الفقه وأصوله ******************************
*******************************************************************************************
vectorizeation ....
training .....
evaluation ....
========== SVM ==========
            Not one     One 
    Not one   736.0   237.0 
        One   153.0  1292.0 
              precision    recall  f1-score   support

     Not one       0.83      0.76      0.79       973
         One       0.84      0.89      0.87      1445

    accuracy                           0.84      2418
   macro avg       0.84      0.83      0.83      2418
weighted avg       0.84      0.84      0.84      2418

Accuracy: 0.84
Precision: 0.84
Recall: 0.89
F1: 0.87
*******************************************************************************************
****************************** result for data القرآن الكريم ****************************

### with Count Vectorizer

# Save_Load Results

In [ ]:
# count_vectorizer_Container_Result = Creat_Container_Result()
# for i in range(len(X_train)):
#     print("*************************************************************************")
#     print("******************************",labels[i],"******************************")
#     print("*************************************************************************")
#     print("vectorizeation ....")
#     X_tr , vectorizer = count_vectorizer(X_train[i])
#     X_te = vectorizer.transform(X_test[i])
#     print("training .....")
#     knn,lr,dt,svm = train_models(X_tr,y_train[i],save_model,labels[i])
#     print('evaluation ....')
#     evaluate_result(knn,X_te,y_test[i],'KNeighborsClassifier',labels[i],count_vectorizer_Container_Result)
#     evaluate_result(lr,X_te,y_test[i],'LogisticRegression',labels[i],count_vectorizer_Container_Result)
#     evaluate_result(dt,X_te,y_test[i],'DecisionTreeClassifier',labels[i],count_vectorizer_Container_Result)
#     evaluate_result(svm,X_te,y_test[i],'SVM',labels[i],count_vectorizer_Container_Result)

In [ ]:
def save_object(obj, filename):
    filename = os.path.join(results_path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename):
    filename = os.path.join(results_path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [ ]:
save_object(TFIDF_Container_Result,"TFIDF_Container_Result")
# save_object(count_vectorizer_Container_Result,"count_vectorizer_Container_Result")
save_object(save_model,"save_model")


# TFIDF_Container_Result = load_object('TFIDF_Container_Result')
# save_model = load_object('save_model')
# load_object(count_vectorizer_Container_Result)

# Select best model 

In [ ]:
best_models = []
for i,label in enumerate(labels):
  best_models.append(save_model['SVM'][label][0])
    # if i<2 or (i>3 and i<8):
    #     best_models.append(save_model['SVM'][label][0])
    # elif i>1 and i<=3:
    #     best_models.append(save_model['KNeighborsClassifier'][label][0])
    # else:
    #     best_models.append(save_model['LogisticRegression'][label][0])

In [ ]:
# save_object(best_models,'best_models__')
# save_object(vectorizers,"vectorizers")

best_models = load_object('best_models__')
vectorizers = load_object('vectorizers')

In [ ]:
new_X_test = []
new_Y_test = []
for i,Y_test in enumerate(y_test):
    x_temp = []
    y_temp = []
    for j,y in enumerate(Y_test):
        if y==1:
            y_temp.append(y)
            x_temp.append(X_test[i][j])
    new_Y_test.append(y_temp) 
    new_X_test.append(x_temp)

In [ ]:
save_object(labels,"labels")
labels

['الفقه وأصوله',
 'القرآن الكريم',
 'الحديث وعلومه',
 'فقه المواريث',
 'العقيدة',
 'فقه الأسرة',
 'الدعوة ووسائلها',
 'الآداب والأخلاق والرقائق']

In [ ]:
for i,X in enumerate(new_X_test):
  j=0
  print("*************************************************************************")
  print("*************************************************************************")
  for j,x in enumerate(X):
    best_pre = []
    print("===================================================================")
    print("Real Label :",labels[i])
    for m,model in enumerate(best_models):
      tx = vectorizers[m].transform([x])
      if model.predict(tx)[0]==1:
        print("Result Model : "+str(labels[m])+" =>",max(model.predict_proba(tx)[0]))
        best_pre.append(max(model.predict_proba(tx)[0]))
      else:
        print("Result Model : "+str(labels[m])+" =>",min(model.predict_proba(tx)[0]))
        best_pre.append(min(model.predict_proba(tx)[0]))
    
    print("***** predict =>",labels[best_pre.index(max(best_pre))],max(best_pre))

Streaming output truncated to the last 5000 lines.
Result Model : فقه المواريث => 0.009406308543169272
Result Model : العقيدة => 0.1124726477327621
Result Model : فقه الأسرة => 0.026424888752090498
Result Model : الدعوة ووسائلها => 0.08494562801315625
Result Model : الآداب والأخلاق والرقائق => 0.8957365960211316
***** predict => الفقه وأصوله 0.9481320143353799
Real Label : الآداب والأخلاق والرقائق
Result Model : الفقه وأصوله => 0.15219887135245125
Result Model : القرآن الكريم => 0.009510967649071849
Result Model : الحديث وعلومه => 0.006790313823134143
Result Model : فقه المواريث => 0.4716127727822556
Result Model : العقيدة => 0.0090782339025201
Result Model : فقه الأسرة => 0.37345263646634264
Result Model : الدعوة ووسائلها => 0.33817587818914663
Result Model : الآداب والأخلاق والرقائق => 0.6649378013474683
***** predict => الآداب والأخلاق والرقائق 0.6649378013474683
Real Label : الآداب والأخلاق والرقائق
Result Model : الفقه وأصوله => 0.027615702844124543
Result Model : القرآن الكريم =>